In [5]:
# Example usage
import json
import torch
from pathlib import Path
from sae_dashboard.neuronpedia.vector_set import VectorSet
# First, create a sample JSON file with a single vector

# initialize a random vector
vector = torch.randn(768)

vector_data = {
    "vectors": [
        vector.tolist()
    ]
}

# Save to JSON file
json_path = Path("sample_vector.json")
with open(json_path, "w") as f:
    json.dump(vector_data, f)

# Load the vector into a VectorSet
vector_set = VectorSet.from_json(
    json_path=json_path,
    d_model=768,  # Example dimension for GPT-2 Small
    hook_point="blocks.0.hook_resid_pre",
    hook_layer=0,
    model_name="gpt2",
    names=["my_custom_vector"]  # Optional custom name
)

# Now you can use the vector set
print(f"Number of vectors: {vector_set.vectors.shape[0]}")
print(f"Vector dimension: {vector_set.vectors.shape[1]}")
print(f"Vector names: {vector_set.names}")

Number of vectors: 1
Vector dimension: 768
Vector names: ['my_custom_vector']


In [6]:
vector_set.vectors.shape

torch.Size([1, 768])

In [7]:

from sae_dashboard.neuronpedia.neuronpedia_vector_runner import (
    NeuronpediaVectorRunner,
    NeuronpediaVectorRunnerConfig,
)

cfg = NeuronpediaVectorRunnerConfig(
    outputs_dir="test_outputs/",
    huggingface_dataset_path="monology/pile-uncopyrighted",
    vector_dtype="float32",
    model_dtype="float32",
    # Small test settings
    n_prompts_total=32,  # Small number for testing
    n_tokens_in_prompt=64,  # Shorter sequences
    n_prompts_in_forward_pass=16,
    n_vectors_at_a_time=1,
    use_wandb=False,  # Disable wandb for testing
)

In [8]:
runner = NeuronpediaVectorRunner(vector_set, cfg)
runner.run()

Device Count: 1
Using specified vector dtype: float32
SAE Device: mps
Model Device: mps
Model Num Devices: 1
Activation Store Device: mps
Dataset Path: monology/pile-uncopyrighted
Forward Pass size: 64
Total number of tokens: 2048
Total number of contexts (prompts): 32
Vector DType: float32
Model DType: float32


/Users/curttigges/miniconda3/envs/sae-d/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

/Users/curttigges/Projects/SAEDashboard/sae_dashboard/neuronpedia/neuronpedia_vector_runner.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokens = torch.load(tokens_

...

/Users/curttigges/Projects/SAEDashboard/sae_dashboard/vector_data_generator.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(


primary acts shape: torch.Size([16, 64, 768]), feature vectors shape: torch.Size([1, 768])


┏━━━━━━┳━━━━━━┳━━━━━━━┓
┃ Task ┃ Time ┃ Pct % ┃
┡━━━━━━╇━━━━━━╇━━━━━━━┩
└──────┴──────┴───────┘

0it [00:00, ?it/s]


AttributeError: 'VectorVisData' object has no attribute 'feature_data_dict'

In [ ]:
output_path = Path("test_outputs/gpt2_hook_resid_pre/batch-0.json")
print(f"Output file exists: {output_path.exists()}")
if output_path.exists():
    with open(output_path) as f:
        data = json.load(f)
        print(f"Generated data for {len(data['features'])} features")